In [1]:
%run ./Utils.ipynb
import os
import cv2
import pyrealsense2 as rs

In [2]:
path = f'../Data/miniexperiment/20.03 (montaz)/Data._basler/SINGLE'

files = [os.path.join(path, x) for x in os.listdir(path)]
#files

In [3]:
mouseX,mouseY = 0,0
img = None
def draw_circle(event,x,y,flags,param):
    global mouseX,mouseY
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img,(x,y),10,(255,0,0),-1)
        mouseX,mouseY = x,y

cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

for file in files:
    img = cv2.imread(file)
    while(1):
        cv2.imshow('image',img)
        k = cv2.waitKey(20) & 0xFF
        if k == 27:
            break
        elif k == ord('a'):
            print(mouseX,mouseY)
cv2.destroyAllWindows()

0 0
0 0


In [15]:
points_floor = [
    [1042, 737, 0],
    [1161, 747, 0],
    [1287, 760, 0],
    [1405, 771, 0],
    [1025, 861, 0],
    [1148, 870, 0],
    [1274, 882, 0],
    [1397, 891, 0],
    [1011, 986, 0],
    [1138, 997, 0],
    [1265, 1007, 0],
    [1392, 1018, 0],
    [996, 1111, 0],
    [1128, 1120, 0],
    [1257, 1128, 0],
    [1382, 1134, 0]
]

points_170 = [
    [598, 91, 170],
    [970, 118, 170],
    [1346, 167, 170],
    [1693, 206, 170],
    [537, 445, 170],
    [931, 489, 170],
    [1323, 536, 170],
    [1685, 576, 170],
    [481, 842, 170],
    [906, 873, 170],
    [1301, 916, 170],
    [1664, 933, 170],
    [444, 1172, 170],
    [895, 1240, 170],
    [1261, 1241, 170],
    [1634, 1278, 170]
]
file = '../Data/miniexperiment/20.03 (montaz)/Data._basler/SINGLE\\Image__2024-03-20__16-55-35.png'
img = cv2.imread(file)

for pf, pa in zip(points_floor, points_170):
    #cv2.circle(img,p,10,(255,0,0),-1)
    cv2.line(img, pf[:2], pa[:2], (255, 0, 0), 1)

img = cv2.resize(img, (0,0), fx=0.6, fy=.6)
cv2.imshow('image',img)
cv2.waitKey()
cv2.destroyAllWindows()


In [5]:
import cv2
import numpy as np

# Example corresponding points (you'll need to replace these with your actual points)
src_points = np.array(points_170, dtype=np.float32)
dst_points = np.array(points_floor, dtype=np.float32)

# Compute homography matrix
#H, _ = cv2.findHomography(src_points, dst_points)
_, T, _ = cv2.estimateAffine3D(src_points, dst_points)
print(T)
# Apply transformation to a point in the air (xa, ya)
xa, ya, za = 1000, 1000, 150  # Example point in the air
transformed_point = cv2.transform(np.array([[[xa, ya, za]]], dtype=np.float32), T)
print(transformed_point[0, 0, 0])
# Extract the projected point on the floor
xf, yf, zf = transformed_point[0][0]

print(f"Projected point on the floor: ({xf:.2f}, {yf:.2f}, {zf:.2f})")

[[ 0.32681159 -0.00814612  4.97798473  0.02928226]
 [-0.00661064  0.33285574  4.18908835  0.0246417 ]
 [ 0.          0.          0.          0.        ]]
1065.3926
Projected point on the floor: (1065.39, 954.63, 0.00)


In [6]:
# Aruco params
aruco_dict = aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_MIP_36h12)
parameters = aruco.DetectorParameters()

parameters.adaptiveThreshConstant = 3 # lower -> more aggresive, default 7
parameters.errorCorrectionRate = 0.2 # default 0.6
parameters.polygonalApproxAccuracyRate = 0.05 # default 0.03
#parameters.useAruco3Detection = 1

detector = aruco.ArucoDetector(aruco_dict, parameters) 

In [14]:
folder = '../Data/miniexperiment/20.03 (montaz)/Data._basler'
height = 160

for i in range(50, 80):
    file = f'7__40069753__20240320_171416501_00{i}.tiff'#f'7__40069753__20240320_170732723_00{i}.tiff'
    img = cv2.imread(os.path.join(folder, file))

    corners, ids, rejected = detector.detectMarkers(img)
    #if ids != None:# and ids[0] == 61:
    aruco.drawDetectedMarkers(img, corners, ids)

    for rect in corners:
    #rect = corners[0]
        x = int(rect[0,:,0].sum()/4)
        y = int(rect[0,:,1].sum()/4)

        transformed_point = cv2.transform(np.array([[[x, y, height]]], dtype=np.float32), T)
        print(transformed_point)
        xt = int(transformed_point[0, 0, 0])
        yt = int(transformed_point[0, 0, 1])
        cv2.line(img, (x, y), (xt, yt), (255, 0, 0), 1)

    img = cv2.resize(img,(0, 0),  fx=0.6, fy=0.6)
    cv2.imshow('image',img)
    cv2.waitKey()
cv2.destroyAllWindows()

[[[1255.1364  976.7255    0.    ]]]
[[[1138.6707  890.5856    0.    ]]]
[[[1104.3987 1036.332     0.    ]]]
[[[1261.8613  875.4514    0.    ]]]
[[[1249.9563   974.83417    0.     ]]]
[[[1140.2721  891.8839    0.    ]]]
[[[1109.9707 1035.554     0.    ]]]
[[[1258.2175  877.5213    0.    ]]]
[[[1244.7761  972.9428    0.    ]]]
[[[1142.2004   893.17566    0.     ]]]
[[[1116.5314 1034.4231    0.    ]]]
[[[1255.2192   879.91077    0.     ]]]
[[[1238.6074   971.40405    0.     ]]]
[[[1143.8181   893.80835    0.     ]]]
[[[1123.1084 1032.6267    0.    ]]]
[[[1253.2095  881.9476    0.    ]]]
[[[1145.1254   893.78186    0.     ]]]
[[[1232.4469  969.5326    0.    ]]]
[[[1129.6935 1030.4973    0.    ]]]
[[[1251.1997  883.9844    0.    ]]]
[[[1146.0896   894.42773    0.     ]]]
[[[1226.605   967.9873    0.    ]]]
[[[1135.633  1028.0483    0.    ]]]
[[[1250.824   885.9881    0.    ]]]
[[[1146.057    895.75916    0.     ]]]
[[[1221.4167  966.4288    0.    ]]]
[[[1140.5923 1025.6191    0.    ]]]
[[[1

In [8]:
folder = '../Data/miniexperiment/20.03 (montaz)/Data._basler'

for i in range(50, 80):
    file = f'7__40069753__20240320_171416501_00{i}.tiff'#f'7__40069753__20240320_170732723_00{i}.tiff'
    path = os.path.join(folder, file)
    img = cv2.imread(path)

    
    img = cv2.resize(img,(0, 0),  fx=0.6, fy=0.6)
    cv2.imshow('image',img)
    key = cv2.waitKey()
    if key == ord('q'):
        break
    elif key == ord('s'):
        print('save', path)
        final_path = os.path.join('../dominik', file)
        os.system(f'copy {path} {final_path}')
cv2.destroyAllWindows()

In [9]:
%run ./Utils.ipynb
import os
import cv2
import pyrealsense2 as rs

In [21]:
file = f'..\Data\miniexperiment/22.03 (1 pokus)\Data_realsense1/20240322_123504.bag'

height = 2650

pipeline = rs.pipeline()
# Create a config object
config = rs.config()

# Tell config that we will use a recorded device from file to be used by the pipeline through playback.
rs.config.enable_device_from_file(config, file)

# Configure the pipeline to stream the depth stream
# Change this parameters according to the recorded bag file resolution
#config.enable_stream(rs.stream.depth, rs.format.z16, 30)

# Start streaming from file
pipeline.start(config)

# Create opencv window to render image in
cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)

# Create colorizer object
colorizer = rs.colorizer()
#frames = pipeline.wait_for_frames()
# Streaming loop
while True:
    # Get frameset of depth
    frames = pipeline.wait_for_frames()
    #print(frames.size())
    # Get depth frame
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    #if not color_frame: continue
    color = np.asanyarray(color_frame.get_data())
    color = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)

    # Create alignment primitive with color as its target stream:
    align = rs.align(rs.stream.color)
    frames = align.process(frames)

    # Update color and depth frames:
    colorized_depth = np.asanyarray(colorizer.colorize(depth_frame).get_data())
    data = np.asanyarray(depth_frame.get_data())
    if data[400, 420] < 2600:
        print(2650 - data[400, 420])
    # Show the two frames together:
    images = np.hstack((color, colorized_depth))


    # Render image in opencv window
    cv2.imshow("Depth Stream", images)
    key = cv2.waitKey()
    # if pressed escape exit program
    if key == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

1624
80
1709
